## Quick Interface Instructions:
* Run the cells below sequentially
* Click a cell to edit
* ** Shift-enter ** to run a cell

## Import useful tools

In [ ]:
# Be patient, this may take a little while
# If you see "In [*]" to the left of this cell, it means it is busy computing, just wait a bit longer
import os, imars3d, numpy as np, glob

In [ ]:
# Be patient, this may take a little while too
# %matplotlib notebook
%matplotlib inline
from matplotlib import pyplot as plt

In [ ]:
import ipywidgets as ipyw
from IPython.display import display

## Define data paths
* In this section we gather information and set up some directories to save the final reconstructed data, as well as some intermediate results. 
* The information needed are 
 - IPTS number
 - sample name

** Please input the experiment IPTS number below **

* Execute the cell, it will prompt you to enter the IPTS number
* In this example, it should be 17413

In [ ]:
ipts = raw_input()
ipts = int(ipts)

** Please input the name of the sample **

In this example, it should be "crack3"

In [ ]:
sample = raw_input()

** Input data directories **

In [ ]:
# use your experiment IPTS number
iptsdir = "/HFIR/CG1DImaging/IPTS-%s/" % ipts

# path to the directory with ct, ob, and df data files or subdirs
datadir = os.path.join(iptsdir,"raw/")

** Output dir for intermediater results **

In [ ]:
# temporary working directory. 
# !!! should be /SNSlocal2/<yourusername>/<experimentname> !!!
import getpass
username = getpass.getuser()
workdir = "/SNSlocal2/%s/%s" % (username, sample)
# !rm -rf {workdir}
if os.path.exists(workdir):
    print "%s already exists" % workdir
    clean = raw_input("remove it?")
    if clean.lower() in ['y', 'yes']:
        !rm -rf {workdir}
else:
    !mkdir -p {workdir}
print "workdir=", workdir

** Output dir for final results **

In [ ]:
# path to save output results (reconstructed slices)
outdir = os.path.join(iptsdir, "shared/processed_data/%s" % sample)
if os.path.exists(outdir):
    print "%s already exists" % outdir
    outdir = raw_input("Please input a path manually. The path should not exist yet: ")
    outdir = os.path.abspath(outdir)
print 'outdir=', outdir

## Information about the CT scan

In [ ]:
ct_info = {}

### CT subdirectory
* CT scan files are stored under one of the subdirectories under IPTS-#####/raw/ct_scans
* Please choose the subdirectory that you want to reconstruct

In [ ]:
ct_scan_root = os.path.join(datadir, 'ct_scans')
ct_scan_subdirs = [d for d in os.listdir(ct_scan_root) if os.path.isdir(os.path.join(ct_scan_root, d))]
assert (ct_scan_subdirs), "No subdir in ct_scans"

ct_scan_select = ipyw.Select(options=ct_scan_subdirs, value=ct_scan_subdirs[0], description="CT scans")
button = ipyw.Button(description='Select')
def on_select(s):
    ct_info['ct_subdir'] = ct_scan_select.value
    print "selected %s" % ct_scan_select.value
button.on_click(on_select)
display(ct_scan_select, button)

### CT filename signature
The CT filenames have a common substring that can be used as a signature.
In the following cell this signature will be calculated. 
Please confirm it is the correct signature so that all relevant CT projection
files will be included in the reconstruction.

In [ ]:
# all files
ct_dir = os.path.join(ct_scan_root, ct_info['ct_subdir'])
ct_files = os.listdir(ct_dir)
# assume all files start with date like 20160918
files_without_dates = [f[9:] for f in ct_files]
# find common prefix
ct_sig0 = os.path.commonprefix(files_without_dates).strip()
ct_sig = '_'.join(ct_sig0.split('_')[:-2])
print "* Example CT scan filenames:"
for f in ct_files[:10]:
    print '  -', f
print
print "* Found signature string for CT to be %r" % ct_sig

In [ ]:
# Please verify that the signature string computed above is correct with the example filenames.
# If it is incorrect, please manually set the value of ct_sig"

# ct_sig = "XXXX"

### Signature of Open beam filenames
* All open beam files are under IPTS-#####/raw/ob.
* Only some OB files are for this sample. Those files have a common signature. Please find that signature and assign it to variable "ob_sig"

In [ ]:
ob_dir = os.path.join(datadir, 'ob')

In [ ]:
# the output of this cell shows all OB files
!ls {ob_dir}

In [ ]:
# find the signature and assign it to ob_sig
# for this example, it is Cracked_3 
ob_sig = "Cracked_3"

In [ ]:
# print the OB files with the given signature
ob_files = [os.path.basename(f) for f in glob.glob(os.path.join(ob_dir, '*%s*.tiff' % ob_sig))]
for f in ob_files: print f

### Signature of dark field filenames
* All dark field files are under IPTS-#####/raw/df.
* Only some DF files are for this sample. Those files have a common signature. Please find that signature and assign it to variable "df_sig"

In [ ]:
df_dir = os.path.join(datadir, 'df')

In [ ]:
# the output of this cell shows all DF files
!ls {df_dir}

In [ ]:
# find the signature and assign it to df_sig
# for this example, it is Cracked_2_DF
df_sig = "Cracked_2_DF"

In [ ]:
# print the DF files with the given signature
df_files = [os.path.basename(f) for f in glob.glob(os.path.join(df_dir, '*%s*.tiff' % df_sig))]
for f in df_files: print f

## Create CT data object

In [ ]:
# import the CT class
from imars3d.CT import CT

In [ ]:
# ask for help
CT?

In [ ]:
# Once the filename patterns are confirmed, 
# you can create the ct object.
ct = CT(datadir, 
        CT_subdir="ct_scans/%s" % ct_info['ct_subdir'], CT_identifier=ct_sig, 
        workdir=workdir, outdir=outdir,
        ob_identifier=ob_sig, df_identifier=df_sig
       )

## Find region of interests

In [ ]:
# compute an average image in order to obtain ROI
# it may take a minute
ave = ct.estimateAverage(ct.ct_series)

In [ ]:
# plot the average image. it may take a few seconds
plt.pcolormesh(ave, cmap="gray")
plt.colorbar()

** For this sample, we can define the ROI as xmin=750, ymin=0, xmax=1300, ymax=2047 **

## Run reconstruction

In [ ]:
%%time
# Now run the reconstruction. 
# Make sure to modify the crop_window and use the numbers obtained from above
# This will take 10-60 minutes.
ct.recon(crop_window=(750,0,1300,2047))

** Check output **

In [ ]:
print outdir

In [ ]:
# recon output
!ls -l {outdir} |head -n 10

In [ ]:
# temporary results
!ls -l {workdir}

In [ ]:
# sinograms
!ls {workdir}/sinogram | head -n 10

** Visualization **

ImageJ. To start, open a terminal and type
```
$ /HFIR/CG1DImaging/shared/Fiji.app/ImageJ-linux64
```